# Implementierung einer Aussagenlogischen Formel in der DNF als Neuronales Netz

## Vorbetrachtung

Satz: Jede aussagenlogische Formel in disjunktiver Normalform kann durch ein zweischichtiges, vorwärtsgetriebenes neuronales Netz realisiert werden.

Zwischenschicht: $z_m$ (m Monome) \
Eingabeschicht: $x_{n} \in \{1,-1\}$ (Aussagenvariablen) \
Ausgabeschicht: Ein Neuron $y$ \
Aktivierungsfunktion: Vorzeichenfunktion $sgn()$

Es folgt: \
$\vec{z} = sgn(w \cdot \vec{x} - \vec{v})$ \
$y = sgn(W \cdot \vec{z} - V)$

Gewichtsmatrizen: $w \in \mathbb{R}^{(m,n)}$ und $W \in \mathbb{R}^{(1,m)}$ \
Schwellwerte: $\vec{v} \in \mathbb{R}^m$ und $V \in \mathbb{R}$

## Initialisierung

### Imports

Numpy wird verwendet um die Matrizen und Vektoren zu speichern und um damit Operationen ausführen zu können.

In [ ]:
import numpy as np

### Konstanten

Als aussagenlogische Formel in DNF wird folgende  gegeben:

$(x_1 \land x_2) \lor (x_3 \land x_4) \lor (x_5 \land x_6) \lor (x_7 \land x_8) \lor (x_9 \land x_{10})$

In [ ]:
# Anzahl Monome
m = 5
# Anzahl Aussagenvariablen
n = 10

### Trainingswerte erstellen

In [ ]:
# Anzahl der zufälligen Belegungen
daten_umfang = 10

# Liste an Aussagenvariablen erstellen
x_array = np.random.choice([1,-1], size=(daten_umfang, n))

def aussagenlogische_formel(x):
    # (x1 und x2) oder (x3 und x4) oder (x5 und x6) oder (x7 und x8) oder (x9 und x10)
    
    y = (
        x[0] == 1 and x[1] == 1 or \
        x[2] == 1 and x[3] == 1 or \
        x[4] == 1 and x[5] == 1 or \
        x[6] == 1 and x[7] == 1 or \
        x[8] == 1 and x[9] == 1
    )

    return 1 if y else -1

# Erwartete Ausgaben erstellen
p_array = np.array([aussagenlogische_formel(i) for i in x_array])

### Aktivierungsfunktion

In [ ]:
def sgn(x):
    return np.where(x < 0, -1, 1)

## Implementierung mit Fehlerrückübertragung

### Gewichte und Schwellwerte

In [ ]:
# Gewichtsmatrizen
# 1. Schicht
w = np.random.rand(m,n)
# 2. Schicht
W = np.random.rand(1,m)

# Schwellwerte
# 1. Schicht
v = np.random.rand(m)
# 2. Schicht
V = np.random.rand(1)

### Training

In [ ]:
# Lernrate
lr = 0.01

for _ in range(1000):
    for i, (x,p) in enumerate(zip(x_array, p_array)):

        print("Epoche: ", i+1)

        # Schritt

        # z = sgn(w*x-v)
        # Berechnung Zwischenschicht
        z = sgn(np.dot(w,x)-v)

        # y = sgn(W*z-V)
        # Berechnung Ausgabeschicht
        y = sgn(np.dot(W,z)-V)

        print(f"Ausgabe: {y}\t Erwartet: {p}")


        # Fehlerrückübertragung
        print("Fehler: ", p-y)

        delta_W = lr * (p-y) * z
        delta_w = lr * (p-y) * np.outer(W, x)

        delta_V = -lr * (p-y)
        delta_v = (-lr * (p-y) * W).squeeze()


        # Delta anwenden

        w += delta_w
        W += delta_W
        v += delta_v
        V += delta_V

## Implementierung mit vorgegebenen Schwellwerten

Die Gewichte für ein DNF müssen folgendermaßen gesetzt werden:

$w_{ij} = 1$ wenn $l_{ij} = x_i$ \
$w_{ij} = -1$ wenn $l_{ij} = \neg x_i$ \
$w_{ij} = 0$ wenn $l_{ij} = w$

$v_i = $  Anzahl Variablen im Monom


alle $W_{ij} = 1$ \
$V = $ 1 - Anzahl der Monome

### Gewichte und Schwellwerte

In [ ]:
# (x1 und x2) oder (x3 und x4) oder (x5 und x6) oder (x7 und x8) oder (x9 und x10)

w = np.array([
    [1,1,0,0,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0,0],
    [0,0,0,0,1,1,0,0,0,0],
    [0,0,0,0,0,0,1,1,0,0],
    [0,0,0,0,0,0,0,0,1,1],
])

v = np.array([
    2,2,2,2,2
])

W = np.ones(shape=(1,m))

V = np.array(1-m)

### Testen

-> Ausgabe entspricht immer dem erwarteten Wert

In [ ]:
for x, p in zip(x_array, p_array):
    # z = sgn(w*x-v)
    # Berechnung Zwischenschicht
    z = sgn(np.dot(w,x)-v)

    # y = sgn(W*z-V)
    # Berechnung Ausgabeschicht
    y = sgn(np.dot(W,z)-V)

    print(f"Ausgabe: {y}\t Erwartet: {p}")

## Vorgegebene Gewichte und Schwellwerte in Fehlerrückübertragung Testen

Der Fehler ist immer gleich null -> das Delta ist gleich null -> Gewichte und Schwellwerte bleiben unverändert.

In [ ]:
for x, p in zip(x_array, p_array):
    # z = sgn(w*x-v)
    # Berechnung Zwischenschicht
    z = sgn(np.dot(w,x)-v)

    # y = sgn(W*z-V)
    # Berechnung Ausgabeschicht
    y = sgn(np.dot(W,z)-V)

    # Fehlerrückübertragung
    print("Fehler: ", p-y)

    delta_W = lr * (p-y) * z
    delta_w = lr * (p-y) * np.outer(W, x)

    delta_V = -lr * (p-y)
    delta_v = (-lr * (p-y) * W).squeeze()